GPT-2 experimenting with Hugging Face
---------------------

Let us import the gpt-2 tokenizer and pre-trained model.

In [2]:
from transformers import GPT2Tokenizer, GPT2Model
import pandas as pd

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tokenization

Let us see how gpt-2 tokenize (pre-process) the text.

In [3]:
# let us use the following text from the tutorial at https://huggingface.co/gpt2
text = "Replace me by any text you'd like."

# let us load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# let us tokenize the text
encoded_input = tokenizer(text, return_tensors = 'pt') # return torch tensor

# print the result
print("Here the decoded tokens")
print([tokenizer.decode(token) for token in encoded_input['input_ids']])

print("\nHere the ids and padding mask")
print(encoded_input)

Here the decoded tokens
["Replace me by any text you'd like."]

Here the ids and padding mask
{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


It tokenize the text with a tokenizer trained on a english vocabulary and give us the corresponding attention mask.

Let us import the extracted sentences and try to tokenize some of them and print the results.

In [4]:
sentences = pd.read_csv("data/extractions/new_data/sent_extraction.csv")

In [5]:
# print a sample 100 sentences
samples = sentences.sample(100, random_state=100).to_dict(orient='index')
samples

{635: {'french_corpus': 'Les routes, les chemins ne sont pas sûrs, il faut sortir armé. Les Ibos du Calabar sont ceux qui ont résisté avec le plus d’acharnement à la pénétration des Européens.',
  'wolof_corpus': 'Ñu dem ba kenn ñemeetul a génn këram te gànnaayuwul ; Ibo yu Kalabaar yi nag, ñoo gënoon a të Tubaab yi.'},
 590: {'french_corpus': 'Au désert, il n’y a pas de guerre.',
  'wolof_corpus': 'Àndandoom yi ñoom, yëguñu sax ni àddina si yépp ngi ci xare bu mag.'},
 152: {'french_corpus': 'Ou peut-être que nous rejetions de cette manière l’autorité excessive de notre père, rendant coup pour coup avec nos bâtons.',
  'wolof_corpus': 'Teewul léeg-léeg mii xel ni ma dañ daan feyu ci gunóor yooyu, ndeysaan, dóor yu metti ya nu Baay daan dóor.'},
 19: {'french_corpus': 'Le sexe des garçons, leur gland rose circoncis.',
  'wolof_corpus': 'Cucu gone yi ñu xarafal ak seen ndéeň lu xonq.'},
 73: {'french_corpus': 'Une force électrique qu’il me fallait accepter, apprivoiser, et pour laquelle

Let us try to tokenize the sentences 101.

In [6]:
# first sentences
example1 = samples[101]

In [7]:
example1

{'french_corpus': 'La mémoire d’un enfant exagère les distances et les hauteurs.',
 'wolof_corpus': 'Gone, su demee bay natt dayo mbaa guddaay, du xam bu ca yem.'}

- On the French corpus

In [8]:
# tokenize the first example
encoded_input = tokenizer(example1['french_corpus'], return_tensors='pt')
print("The encoded tokens with their mask")
print(encoded_input)
print("\nThe decoded tokens")
[tokenizer.decode(token) for token in encoded_input['input_ids'][0]]

The encoded tokens with their mask
{'input_ids': tensor([[14772,   285,  2634,  5908,   557,   288,   447,   247,   403,   551,
            69,   415,   409,   363, 35979, 10287, 18868,  2123, 10287,   387,
          1133,  1834,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

The decoded tokens


['La',
 ' m',
 'é',
 'mo',
 'ire',
 ' d',
 '�',
 '�',
 'un',
 ' en',
 'f',
 'ant',
 ' ex',
 'ag',
 'ère',
 ' les',
 ' distances',
 ' et',
 ' les',
 ' ha',
 'ute',
 'urs',
 '.']

- On the Wolof corpus

In [9]:
# tokenize the first example
encoded_input = tokenizer(example1['wolof_corpus'], return_tensors='pt')
print("The encoded tokens with their mask")
print(encoded_input)
print("\nThe decoded tokens")
[tokenizer.decode(token) for token in encoded_input['input_ids'][0]]

The encoded tokens with their mask
{'input_ids': tensor([[   38,   505,    11,   424,  1357,  1453, 15489,   299,  1078,  1110,
            78,   285,  7012,    64,   308,  4185,    64,   323,    11,  7043,
          2124,   321,   809,  1275,   331,   368,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

The decoded tokens


['G',
 'one',
 ',',
 ' su',
 ' dem',
 'ee',
 ' bay',
 ' n',
 'att',
 ' day',
 'o',
 ' m',
 'ba',
 'a',
 ' g',
 'udd',
 'a',
 'ay',
 ',',
 ' du',
 ' x',
 'am',
 ' bu',
 ' ca',
 ' y',
 'em',
 '.']

We don't obtain what we expected because the tokenizer are not trained on a french corpus or a wolof corpus.

#### Improve the tokenization with a model

In [10]:
# import necessary libraries
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

import random

random.seed(100)

Let us improve the tokenization of the texts by training a tokenizer from scratch. The tutorial is available at the following link [Training_Tokenizer_from_scratch](https://huggingface.co/learn/nlp-course/chapter6/8?fw=pt).

- First, we must load batch of sentences. Let us say that we will a batch of 15 sentences at a time.

In [11]:
# recuperate the sentences as list of sentences (simple method)
french_sentences = sentences['french_corpus'].to_list()

wolof_sentences = sentences['wolof_corpus'].to_list()

# Define Batch sizes
BATCH_SIZE = 5

# let us use generators to load the french and wolof sentences
def load_french_sentences():
    
    for i in range(0, len(french_sentences), BATCH_SIZE):
        
        yield french_sentences[i : i + BATCH_SIZE]

def load_wolof_sentences():
    
    for j in range(0, len(wolof_sentences), BATCH_SIZE):
        
        yield wolof_sentences[j : j + BATCH_SIZE]


##### BPE (Byte-Pair Encoding tokenization) tokenizer

- Let us load the BPE tokenizer

In [12]:
bpe_tokenizer = Tokenizer(models.BPE())

- Configure a pre-tokenizer

In [119]:
# add a byte level pre-tokenizer and specify that we will not a space to the first word
bpe_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

- Test the pre-tokenizer

In [120]:
# let us test the pre-tokenizer with a random sentence
bpe_tokenizer.pre_tokenizer.pre_tokenize_str(french_sentences[0])

[('Tout', (0, 4)),
 ('ĠÃªtre', (4, 9)),
 ('Ġhumain', (9, 16)),
 ('Ġest', (16, 20)),
 ('Ġle', (20, 23)),
 ('ĠrÃ©sultat', (23, 32)),
 ('Ġd', (32, 34)),
 ('âĢĻ', (34, 35)),
 ('un', (35, 37)),
 ('ĠpÃ¨re', (37, 42)),
 ('Ġet', (42, 45)),
 ('Ġune', (45, 49)),
 ('ĠmÃ¨re', (49, 54)),
 ('.', (54, 55))]

- If using the decoder part of the transformer is the GPT-2 model then we need a end-to-text token. Let us initialize the bpe trainer with a vocab size of `10000`.

In [16]:
bpe_trainer = trainers.BpeTrainer(vocab_size = 10000, special_tokens = ["<|endoftext|>"])

- Let us add the batch iterator to the trainer and test on a sample

Let us test for the french corpus and after with the wolof corpus.

1. On the french corpus

In [123]:
bpe_tokenizer.train_from_iterator(load_french_sentences(), trainer=bpe_trainer)

Let us use a decoder to decode the ids.

In [124]:
bpe_tokenizer.decoder = decoders.ByteLevel()

In [125]:
# tokenize a sample (let us take 10 sentences) and print the results
for i in range(10):
    
    sentence = random.choice(french_sentences) 
    
    print(f"For the following sentence:\n{sentence}")
    
    print("We obtain the following:")
    
    french_encoding = bpe_tokenizer.encode(sentence)
    
    print(f"- Tokens: {french_encoding.tokens}")
    
    print(f"- Ids: {french_encoding.ids}")
    
    print(f"- Attention masks: {french_encoding.attention_mask}")
    
    print(f"- Decoded tokens: {bpe_tokenizer.decode(french_encoding.ids)}")

    print("-------------")
    

For the following sentence:
Nous frappions à nouveau, jusqu’à en avoir mal aux mains, comme si nous combattions un ennemi invisible.
We obtain the following:
- Tokens: ['Nous', 'Ġfrappions', 'ĠÃł', 'Ġnouveau', ',', 'Ġjusqu', 'âĢĻ', 'Ãł', 'Ġen', 'Ġavoir', 'Ġmal', 'Ġaux', 'Ġmains', ',', 'Ġcomme', 'Ġsi', 'Ġnous', 'Ġcombattions', 'Ġun', 'Ġennemi', 'Ġinvisible', '.']
- Ids: [569, 2286, 137, 1134, 5, 486, 103, 129, 147, 1596, 450, 377, 1087, 5, 303, 403, 262, 8790, 185, 4249, 3049, 7]
- Attention masks: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
- Decoded tokens: Nous frappions à nouveau, jusqu’à en avoir mal aux mains, comme si nous combattions un ennemi invisible.
-------------
For the following sentence:
Cela va de la frontière avec le Cameroun sous mandat français, au sud-est, jusqu’aux confins de l’Adamawa au nord, et comprend la plus grande partie des chefferies et des petits royaumes qui ont échappé à l’autorité directe de l’Angleterre après le départ des Allem

We can test it on a pair of sentences as follows (for 2 sentences):

In [126]:
random.seed(100)

sentences = [random.choice(french_sentences) for i in range(2)]

encoding = bpe_tokenizer.encode(*sentences)

print(sentences)
print(encoding.tokens)
print(encoding.ids)

['Nous frappions à nouveau, jusqu’à en avoir mal aux mains, comme si nous combattions un ennemi invisible.', 'Cela va de la frontière avec le Cameroun sous mandat français, au sud-est, jusqu’aux confins de l’Adamawa au nord, et comprend la plus grande partie des chefferies et des petits royaumes qui ont échappé à l’autorité directe de l’Angleterre après le départ des Allemands : Kantu, Abong, Nkom, Bum, Foumban, Bali.']
['Nous', 'Ġfrappions', 'ĠÃł', 'Ġnouveau', ',', 'Ġjusqu', 'âĢĻ', 'Ãł', 'Ġen', 'Ġavoir', 'Ġmal', 'Ġaux', 'Ġmains', ',', 'Ġcomme', 'Ġsi', 'Ġnous', 'Ġcombattions', 'Ġun', 'Ġennemi', 'Ġinvisible', '.', 'Cela', 'Ġva', 'Ġde', 'Ġla', 'ĠfrontiÃ¨re', 'Ġavec', 'Ġle', 'ĠCameroun', 'Ġsous', 'Ġmandat', 'ĠfranÃ§ais', ',', 'Ġau', 'Ġsud', '-', 'est', ',', 'Ġjusqu', 'âĢĻ', 'aux', 'Ġconfins', 'Ġde', 'Ġl', 'âĢĻ', 'Adamawa', 'Ġau', 'Ġnord', ',', 'Ġet', 'Ġcomprend', 'Ġla', 'Ġplus', 'Ġgrande', 'Ġpartie', 'Ġdes', 'Ġchefferies', 'Ġet', 'Ġdes', 'Ġpetits', 'Ġroyaumes', 'Ġqui', 'Ġont', 'ĠÃ©chappÃ©

In [127]:
bpe_tokenizer.get_vocab_size()

9550

We have 9550 tokens in the French corpus.

- On the Wolof corpus (For the next we will do this separately for each corpus)

In [128]:
bpe_tokenizer.train_from_iterator(load_wolof_sentences(), trainer=bpe_trainer)


In [129]:
# tokenize a sample (let us take 10 sentences) and print the results
for i in range(10):
    
    sentence = random.choice(wolof_sentences) 
    
    print(f"For the following sentence:\n{sentence}")
    
    print("We obtain the following:")
    
    wolof_encoding = bpe_tokenizer.encode(sentence)
    
    print(f"- Tokens: {wolof_encoding.tokens}")
    
    print(f"- Ids: {wolof_encoding.ids}")
    
    print(f"- Attention masks: {wolof_encoding.attention_mask}")
    
    print(f"- Decoded tokens: {bpe_tokenizer.decode(wolof_encoding.ids)}")

    print("-------------")
    

For the following sentence:
Ci weeru mars 1932 la samay way-jur jóge Forestry House ca Bamendaa, dem dëkki Bansoo, fa nguur gi waroon a tabax ab fajukaay.
We obtain the following:


PanicException: no entry found for key

We see an error indicating that some wolof tokens are not identified since we already had trained the tokenizer on the french corpus and obtained 10000 when adding some wolof tokens. Let us train another tokenizer for the wolof corpus.

In [17]:
bpe_tokenizer2 = Tokenizer(models.BPE())

# add a byte level pre-tokenizer and specify that we will not a space to the first word
bpe_tokenizer2.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

bpe_tokenizer2.train_from_iterator(load_wolof_sentences(), trainer=bpe_trainer)

bpe_tokenizer2.decoder = decoders.ByteLevel()

In [18]:
# tokenize a sample (let us take 10 sentences) and print the results
for i in range(10):
    
    sentence = random.choice(wolof_sentences) 
    
    print(f"For the following sentence:\n{sentence}")
    
    print("We obtain the following:")
    
    wolof_encoding = bpe_tokenizer2.encode(sentence)
    
    print(f"- Tokens: {wolof_encoding.tokens}")
    
    print(f"- Ids: {wolof_encoding.ids}")
    
    print(f"- Attention masks: {wolof_encoding.attention_mask}")
    
    print(f"- Decoded tokens: {bpe_tokenizer2.decode(wolof_encoding.ids)}")

    print("-------------")
    

For the following sentence:
Bu ñu agsee ci dëkk, ndawal buur bee leen di teerusi, dalal leen ca pénc ma, ñu portalewu ak buur beek i dagam.
We obtain the following:
- Tokens: ['Bu', 'ĠÃ±u', 'Ġagsee', 'Ġci', 'ĠdÃ«kk', ',', 'Ġndawal', 'Ġbuur', 'Ġbee', 'Ġleen', 'Ġdi', 'Ġteerusi', ',', 'Ġdalal', 'Ġleen', 'Ġca', 'ĠpÃ©nc', 'Ġma', ',', 'ĠÃ±u', 'Ġportalewu', 'Ġak', 'Ġbuur', 'Ġbeek', 'Ġi', 'Ġdagam', '.']
- Ids: [1269, 154, 3381, 116, 239, 4, 3241, 1330, 1100, 205, 133, 3501, 4, 1353, 205, 189, 3634, 126, 4, 154, 6139, 144, 1330, 958, 175, 4568, 6]
- Attention masks: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
- Decoded tokens: Bu ñu agsee ci dëkk, ndawal buur bee leen di teerusi, dalal leen ca pénc ma, ñu portalewu ak buur beek i dagam.
-------------
For the following sentence:
Ñax mi muur joor gi dafa bare ba may fàttali géej, xóot te raglu ni moom.
We obtain the following:
- Tokens: ['Ãĳax', 'Ġmi', 'Ġmuur', 'Ġjoor', 'Ġgi', 'Ġdafa', 'Ġbare', 'Ġba', 'Ġmay',

In [20]:
bpe_tokenizer2.get_vocab_size()

7034

We have 7034 tokens into the wolof vocabulary.